In [ ]:
import timeit
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
from functools import reduce
import operator

In [1]:
from jax import tree_util
from graphax.sparse.block import BlockSparseTensor

def flatten_block_sparse_tensor(tensor):
    children = (tensor.blocks,)
    aux_data = (tensor.out_dims, tensor.primal_dims, tensor.out_shape, tensor.primal_shape, tensor.elementary_block_idx, tensor.pre_transforms, tensor.post_transforms)
    return children, aux_data

def unflatten_block_sparse_tensor(aux_data, children):
    blocks, = children
    out_dims, primal_dims, out_shape, primal_shape, elementary_block_idx, pre_transforms, post_transforms = aux_data
    return BlockSparseTensor(out_dims, primal_dims, out_shape, primal_shape, blocks, elementary_block_idx, pre_transforms, post_transforms)

tree_util.register_pytree_node(BlockSparseTensor, flatten_block_sparse_tensor, unflatten_block_sparse_tensor)

In [2]:
from jax import jit, make_jaxpr
from graphax.sparse.block import BlockSparseTensor, SparseDimension, new_block_sparse_tensor
import jax.random as jrand

block_nums = 10
block_size = 10
k1, k2 = jrand.split(jrand.PRNGKey(42), 2)
x_blocks = jrand.normal(k1, (block_nums, block_size, block_size))
y_blocks = jrand.normal(k2, (block_nums, block_size, block_size))
x = new_block_sparse_tensor([SparseDimension(0, block_nums, 0, 1, 0)], [SparseDimension(1, block_nums, 0, 0, 1)], x_blocks)
print("###")
y = new_block_sparse_tensor([SparseDimension(0, block_nums, 0, 1, 0)], [SparseDimension(1, block_nums, 0, 0, 1)], y_blocks)
print("---")

def matmul(a, b):
    return a @ b

jit_matmul = jit(matmul)

jaxpr = make_jaxpr(jit_matmul)(x, y)
# print(jit_matmul(x,y))
print(jaxpr)


###
---
{ lambda ; a:f32[10,10,10] b:f32[10,10,10]. let
    c:f32[10,10,10] = pjit[
      name=matmul
      jaxpr={ lambda ; d:f32[10,10,10] e:f32[10,10,10]. let
          f:f32[10,10,10] = dot_general[
            dimension_numbers=(([2], [1]), ([0], [0]))
            preferred_element_type=float32
          ] d e
        in (f,) }
    ] a b
  in (c,) }


In [3]:
res = (x@y).dense()
dense_res=x.dense()@y.dense()

In [4]:
import jax.numpy as jnp
eq=jnp.isclose(res, dense_res)
eq.all()

Array(True, dtype=bool)

In [ ]:
delta = abs(res - dense_res).flatten()

In [ ]:
[x for x in abs(res - dense_res).flatten() if x != 0]

In [ ]:
import json
import os
import ast

res = {}
for run in os.listdir('data'):
    with open('data/' + run, "r") as f:
        cur = json.load(f)
    
    name = run.split('.', 2)[0]
    res_d1 = res[name] = {}
    
    for key1, val1 in list(cur.items()):
        key1 = ast.literal_eval(key1)
        res_d2 = res_d1[key1] = {}
        
        for key2, val2 in list(val1.items()):
        
            if name.endswith('_mul_sum'):
                val2 = tuple(val2)
            key2 = ast.literal_eval(key2)
            res_d2[key2] = val2

In [ ]:
def get_title(key: str):
    return ' '.join([x if x.isupper() else x.capitalize() for x in key.split('_') ])

In [ ]:
first_layer_keys = res.keys()
mul_sum_keys = set(key for key in first_layer_keys if key.endswith('mul_sum'))
exp_data_keys = first_layer_keys - mul_sum_keys
stat_operator = np.median

In [ ]:
# Test memory at some point maybe even test time
# Test on CPU

In [ ]:
block_shape_to_plot = (100, 100)
blocks = {key: [] for key in exp_data_keys}
times = {key: [] for key in exp_data_keys}

for key in exp_data_keys:
    for non_block_shape, block_shapes in res[key].items():
        if block_shape_to_plot in block_shapes:
            blocks[key].append(reduce(operator.mul, non_block_shape))
        times[key].append(stat_operator(block_shapes[block_shape_to_plot]))
    b, t = zip(*sorted(zip(blocks[key], times[key])))
    blocks[key] = np.array(b)
    times[key] = np.array(t)

plt.figure(figsize=(10, 6))

for key in exp_data_keys:
    plt.plot(blocks[key], times[key], marker='o', label=get_title(key))

plt.xlabel("Number of Blocks")
plt.ylabel("Time (seconds)")
plt.title(f"Time vs. Number of Blocks\n(Block shape = {block_shape_to_plot})")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
block_shape_to_plot = (100,100)
blocks = {key: [] for key in mul_sum_keys}
times = {key: [] for key in mul_sum_keys}

for key in mul_sum_keys:
    for non_block_shape, block_shapes in res[key].items():
        if block_shape_to_plot in block_shapes:
            blocks[key].append(reduce(operator.mul, non_block_shape))
            times[key].append(np.sum(block_shapes[block_shape_to_plot]))
    b, t = zip(*sorted(zip(blocks[key], times[key])))
    blocks[key] = np.array(b)
    times[key] = np.array(t)

plt.figure(figsize=(10, 6))

for key in mul_sum_keys:
    plt.plot(blocks[key], times[key], label=get_title(key.split('_',2)[0]))

plt.xlabel("Number of Blocks")
plt.ylabel("Operations")
plt.title(f"Operations vs. Number of Blocks\n(Block shape = {block_shape_to_plot})")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
non_block_shape_to_plot = (100,)
blocks = {key: [] for key in exp_data_keys}
times = {key: [] for key in exp_data_keys}

for key in exp_data_keys:
    if non_block_shape_to_plot in res[key]:
        block_shapes = res[key][non_block_shape_to_plot]
        for block_shape in block_shapes:
            blocks[key].append(reduce(operator.mul, block_shape))
            times[key].append(stat_operator(block_shapes[block_shape]))
    b, t = zip(*sorted(zip(blocks[key], times[key])))
    blocks[key] = np.array(b)
    times[key] = np.array(t)
        


plt.figure(figsize=(10, 6))

for key in exp_data_keys:
    plt.plot(blocks[key], times[key], marker='o', label=get_title(key))
    
plt.xlabel("Block Size")
plt.ylabel("Time (seconds)")
plt.title(f"Time vs. Block Size\n(Non-Blocks = {non_block_shape_to_plot})")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
non_block_shape_to_plot = (100,)
blocks = {key: [] for key in mul_sum_keys}
times = {key: [] for key in mul_sum_keys}

for key in mul_sum_keys:
    if non_block_shape_to_plot in res[key]:
        block_shapes = res[key][non_block_shape_to_plot]
        for block_shape in block_shapes:
            blocks[key].append(reduce(operator.mul, block_shape))
            times[key].append(stat_operator(block_shapes[block_shape]))
    b, t = zip(*sorted(zip(blocks[key], times[key])))
    blocks[key] = np.array(b)
    times[key] = np.array(t)


plt.figure(figsize=(10, 6))

for key in mul_sum_keys:
    plt.plot(blocks[key], times[key], label=get_title(key.split('_',2)[0]))
    
plt.xlabel("Block Size")
plt.ylabel("Operations")
plt.title(f"Operations vs. Block Size\n(Blocks = {non_block_shape_to_plot})")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

heatmap_data = {key: [] for key in exp_data_keys}   
non_block_shapes_labels = sorted(next(iter(res.values())).keys())
block_shapes_labels = sorted(next(iter(next(iter(res.values())).values())))

for non_block_shapes in non_block_shapes_labels:
    rows = {key: [] for key in exp_data_keys}
    for block_shape in block_shapes_labels:
        for key in exp_data_keys:
            if block_shape in res[key][non_block_shapes]:
                rows[key].append(stat_operator(res[key][non_block_shapes][block_shape]))
            else:
                rows[key].append(np.nan)
    for key in exp_data_keys:
        heatmap_data[key].append(rows[key])
for key in exp_data_keys:
    heatmap_data[key] = np.array(heatmap_data[key])
    
fig, axes = plt.subplots(-(len(exp_data_keys)//-2), 2, figsize=(14, 12))

vmax = min([np.quantile(heatmap_data[key], 0.8) for key in exp_data_keys])
vmin = max([np.quantile(heatmap_data[key], 0.2) for key in exp_data_keys])

for i, key in enumerate(exp_data_keys):
    if len(exp_data_keys) >= 2:
        idx = [i // 2, i % 2]
    else:
        idx = [i]
    
    sns.heatmap(
        heatmap_data[key],
        ax=axes[*idx],
        xticklabels=block_shapes_labels,
        yticklabels=non_block_shapes_labels,
        vmin=vmin,
        vmax=vmax,
        norm=plt.matplotlib.colors.LogNorm(),
        cmap="viridis",
        cbar_kws={"label": "Time (seconds)"}
    )
    axes[*idx].set_xlabel("Non-Block Shape")
    axes[*idx].set_ylabel("Block Shape")
    axes[*idx].set_title(get_title(key))

if len(exp_data_keys) % 2:
    if len(exp_data_keys) >= 2:
        last_idx = [-(len(exp_data_keys)//-2) - 1, 1]
    else:
        last_idx = [1]
    axes[*last_idx].axis('off')

plt.tight_layout()

plt.show()

In [ ]:
data = {key: {'time': [], 'ops': []} for key in exp_data_keys}  
for key1 in exp_data_keys:
    for key2, val2 in res[key1].items():
        for key3, val3 in val2.items():
            if key1.endswith('sparse'):
                if 'sparse_mul_sum' in res[key1]:
                    continue
            if key1.endswith('sparse'):
                ops = sum(res['sparse_mul_sum'][key2][key3])
            else:
                ops = sum(res['dense_mul_sum'][key2][key3])
            data[key1]['ops'].append(ops)
            data[key1]['time'].append(stat_operator(val3))

plt.figure(figsize=(10, 6))

for key in exp_data_keys:
    plt.scatter(data[key]['ops'], data[key]['time'], label=key.replace('_',' ').title(), marker="o")

# Customize the plot
plt.title("Median Runtimes vs Operations")
plt.xlabel("Number of Operations")
plt.ylabel("Median Runtime")
plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.grid(True)
plt.show()